<a href="https://colab.research.google.com/github/asampat3090/musicalai/blob/main/_5_musicgpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MusicGPT (Step 2)
In this notebook we will attempt to use the GPT-2 decoder-only architecture to generate audio sequences that continue the input, that is we will ask it to predict the next audio token in an auto-regressive way. We expect this will not be as good as architectures like [MusicLM](https://google-research.github.io/seanet/musiclm/examples/) or [MusicGen](https://ai.honu.io/papers/musicgen/) where text-conditioning is used, but we hope it clarifies how a simple model can be built to continue audio tokens. This will be trained and evaluated on open-source audio sample datasets.

Basic implementation of the GPT-2 architecture. We will be modifying this to take in sequential tokenized audio inputs, but will implement the architecture first to recap the structure and validate the choices made. We will revisit the shape of `x` and hyperparameters present in `config` (i.e. should we use the same params as GPT-2?)

Full Code:

## Building Blocks

We will start with the basic model building blocks of the architecture of the transformer architecture with appropriate hyperparameters which we will update when we need to shanve the input and embedding type.

In [ ]:
import math
import inspect
import dataclasses as dataclass

import torch
import torch.nn as nn
from torch.nn import functional as F

### LayerNorm

Unlike batch norm layer norm does not impose constraints on the size of the mini-batch and can be used in a pure online way batch size 1. More specifically we compute the mean and standard deviation over all hidden units in a layer (rather than across across all N in the batch in batch norm)

$\mu^l = \frac{1}{H}\sum_{i=1}^{H} a_i^l$

$\sigma^l=\sqrt{\frac{1}{H}\sum_{i=1}^{H}(a_i^l -\mu^l)^2}$

More details: https://paperswithcode.com/method/layer-normalization

In [ ]:
# Adapted from https://github.com/karpathy/nanoGPT/blob/master/model.py

class LayerNorm(nn.Module):
    """this module is already defined in pytorch but we add a bias term"""

    def __init__(self, ndim, bias):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(ndim))
        self.bias = nn.params(torch.zeroes(ndim)) if bias else None

    def forward(self,input):
        return F.layer_norm(input,  self.weight.shape,self.weight,self.bias,1e-5)

### Causal Self Attention (Masked Multihead Attention)

Here we are using the GPT-2 architecture which is a decoder only network that uses causal self-attention, that is the queries, keys and values all come from the same input sequence and it is causal because it will only consider all tokens up to and including the current context but ignore anything after that. At each subsequent step it will consider one more token. This decoder is auto-regressive so will step through token by token to generate a result.

In [ ]:
# Adapted from https://github.com/karpathy/nanoGPT/blob/master/model.py

class CausalSelfAttention(nn.Module):

    def __init__(self, config): # config is a dict of values to be defined
        super().__init__()
        assert config.n_embd % config.n_head == 0 # n_embd = d_e = d_k = d_q = d_v
        # key, query, value projections for all heads, but in a batch
        self.c_attn = nn.Linear(config.n_embd, 3*config.n_embd, bias=config.bias)
        # output projection
        self.c_proj = nn.Linear(config.n_embd, config.n_embd, bias=config.bias)
        # regularization
        self.attn_dropout = nn.Dropout(config.dropout)
        self.resid_dropout = nn.Dropout(config.dropout)
        # define params needed for downstream processing
        self.n_head = config.n_head
        self.n_embd = config.n_embd
        self.dropout = config.dropout # probability of dropout
        # causal mask to ensure that attention is only applied to the left in the input sequence
        self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size))
                                        .view(1, 1, config.block_size, config.block_size))

    def forward(self, x):
        B, T, C = x.size() #batch size, sequence length, embedding dim (n_embd)

        # calculate query, key, value for all heads in batch
        q, k, v = self.c_attn(x).split(self.n_embd, dim=2)
        # move head forward to be the batch dim -> note embeddings are split across heads for multi-head
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)

        # causal self-attention; (B, nh, T, hs) x (B, nh, T, hs) -> (B, nh, T, T)
        attn = (q @ k.transpose(-2,-1)) * (1.0/math.sqrt(k.size(-1))) # (B,nh,T,hs) X (B,nh,hs,T) -> (B,nh,T,T)
        attn = attn.masked_fill(self.bias[:,:,:T,:T] == 0, float('-inf')) # fill in top right corner of matrix (ignore subsequent tokens)
        attn = F.softmax(attn, dim=-1)
        attn = self.attn_dropout(attn)
        y = attn @ v # (B,nh,T,T) x (B,nh,T,hs) -> (B,nh,T,hs)
        # reassemble multiple heads into one (B,nh,T,hs) -> (B,T,C)
        y = y.transpose(1,2).contiguous().view(B,T,C) # combine nh and hs to reform C
        # apply final projection and dropout
        y = self.resid_dropout(self.c_proj(y))
        return y

We will run a basic test to confirm this works, using a `1x3x3` tensor, that is `B=1`, `T=3` and `C=3`

In [ ]:
x = torch.tensor([[[0.2,-0.1,-0.9],[0.1,-0.4,-0.9],[0.5,-0.3,-0.9]]]) # 1 x 3 x 3
config = {
    "n_head": 3,
    "n_embd": 3,
    "block_size": 3,
    "bias": False,
    "dropout": 0.5
}
class DotDict(dict):
    """
    a dictionary that supports dot notation
    as well as dictionary access notation
    usage: d = DotDict() or d = DotDict({'val1':'first'})
    set attributes: d.val2 = 'second' or d['val2'] = 'second'
    get attributes: d.val2 or d['val2']
    """
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

test_CSA = CausalSelfAttention(DotDict(config))
print(test_CSA.forward(x).shape)

torch.Size([1, 3, 3])


### MLP - Multilayer Perceptron

In [ ]:
# Adapted from https://github.com/karpathy/nanoGPT/blob/master/model.py

class MLP(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.c_fc = nn.Linear(config.n_embd, 4*config.n_embd, bias=config.bias)
        self.gelu = nn.GELU()
        self.c_proj = nn.Linear(4*config.n_embd, config.n_embd, bias=config.bias)
        self.dropout = nn.Dropout(config.dropout)

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        x = self.dropout(x)
        return x

### Composite Block for GPT-2

Now we can compose the block used in GPT-2. The decoder block takes both positional and token embeddings, layer normalizes, passes through the masked self-attention, layer normalizes, and finally runs through an MLP layer. More details are illustrated well here: http://jalammar.github.io/illustrated-gpt2/

In the final model we can repeat this block `n_layer` number of times.

In [ ]:
class Block(nn.Module):

    def __init__(self, config):
        self.ln_1 = LayerNorm(config.n_embd, bias=config.bias)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = LayerNorm(config.n_embd, bias=config.bias)
        self.mlp = MLP(config)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x

### GPT Model (adapted for music) - MusicGPT

Here we will adapt the nanoGPT aka GPT-2 architecture to work with encoded audio with multiple types of indices. The goal is still to predict the next sequence in time, though we will have to be clever with the encoding and decoding of these encodings.

We will also need to come up with a "word token embedding" and "word positional embedding" which works for audio. We will call these "audio token embedding" and "audio positional embedding" which will be attempt to find an equivalent representation for each time step in the audio (similar to words or other tokens in language)

In [ ]:
# class MusicGPT(nn.Module):
#     pass

In [ ]:
class GPT(nn.Module):

    def __init__(self, config):
        super().__init__()
        assert config.vocab_size is not None
        assert config.block_size is not None
        self.config = config

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd),
            wpe = nn.Embedding(config.block_size, config.n_embd),
            drop = nn.Dropout(config.dropout),
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f = LayerNorm(config.n_embd, bias=config.bias),
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
        # with weight tying when using torch.compile() some warnings get generated:
        # "UserWarning: functional_call was passed multiple values for tied weights.
        # This behavior is deprecated and will be an error in future versions"
        # not 100% sure what this is, so far seems to be harmless. TODO investigate
        self.transformer.wte.weight = self.lm_head.weight # https://paperswithcode.com/method/weight-tying

        # init all weights
        self.apply(self._init_weights)
        # apply special scaled init to the residual projections, per GPT-2 paper
        for pn, p in self.named_parameters():
            if pn.endswith('c_proj.weight'):
                torch.nn.init.normal_(p, mean=0.0, std=0.02/math.sqrt(2 * config.n_layer))

        # report number of parameters
        print("number of parameters: %.2fM" % (self.get_num_params()/1e6,))

    def get_num_params(self, non_embedding=True):
        """
        Return the number of parameters in the model.
        For non-embedding count (default), the position embeddings get subtracted.
        The token embeddings would too, except due to the parameter sharing these
        params are actually used as weights in the final layer, so we include them.
        """
        n_params = sum(p.numel() for p in self.parameters())
        if non_embedding:
            n_params -= self.transformer.wpe.weight.numel()
        return n_params

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        device = idx.device
        b, t = idx.size()
        assert t <= self.config.block_size, f"Cannot forward sequence of length {t}, block size is only {self.config.block_size}"
        pos = torch.arange(0, t, dtype=torch.long, device=device) # shape (t)

        # forward the GPT model itself
        tok_emb = self.transformer.wte(idx) # token embeddings of shape (b, t, n_embd)
        pos_emb = self.transformer.wpe(pos) # position embeddings of shape (t, n_embd)
        x = self.transformer.drop(tok_emb + pos_emb)
        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)

        if targets is not None:
            # if we are given some desired targets also calculate the loss
            logits = self.lm_head(x)
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1)
        else:
            # inference-time mini-optimization: only forward the lm_head on the very last position
            logits = self.lm_head(x[:, [-1], :]) # note: using list [-1] to preserve the time dim
            loss = None

        return logits, loss

    def crop_block_size(self, block_size):
        # model surgery to decrease the block size if necessary
        # e.g. we may load the GPT2 pretrained model checkpoint (block size 1024)
        # but want to use a smaller block size for some smaller, simpler model
        assert block_size <= self.config.block_size
        self.config.block_size = block_size
        self.transformer.wpe.weight = nn.Parameter(self.transformer.wpe.weight[:block_size])
        for block in self.transformer.h:
            if hasattr(block.attn, 'bias'):
                block.attn.bias = block.attn.bias[:,:,:block_size,:block_size]

    @classmethod
    def from_pretrained(cls, model_type, override_args=None):
        assert model_type in {'gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl'}
        override_args = override_args or {} # default to empty dict
        # only dropout can be overridden see more notes below
        assert all(k == 'dropout' for k in override_args)
        from transformers import GPT2LMHeadModel
        print("loading weights from pretrained gpt: %s" % model_type)

        # n_layer, n_head and n_embd are determined from model_type
        config_args = {
            'gpt2':         dict(n_layer=12, n_head=12, n_embd=768),  # 124M params
            'gpt2-medium':  dict(n_layer=24, n_head=16, n_embd=1024), # 350M params
            'gpt2-large':   dict(n_layer=36, n_head=20, n_embd=1280), # 774M params
            'gpt2-xl':      dict(n_layer=48, n_head=25, n_embd=1600), # 1558M params
        }[model_type]
        print("forcing vocab_size=50257, block_size=1024, bias=True")
        config_args['vocab_size'] = 50257 # always 50257 for GPT model checkpoints
        config_args['block_size'] = 1024 # always 1024 for GPT model checkpoints
        config_args['bias'] = True # always True for GPT model checkpoints
        # we can override the dropout rate, if desired
        if 'dropout' in override_args:
            print(f"overriding dropout rate to {override_args['dropout']}")
            config_args['dropout'] = override_args['dropout']
        # create a from-scratch initialized minGPT model
        config = GPTConfig(**config_args)
        model = GPT(config)
        sd = model.state_dict()
        sd_keys = sd.keys()
        sd_keys = [k for k in sd_keys if not k.endswith('.attn.bias')] # discard this mask / buffer, not a param

        # init a huggingface/transformers model
        model_hf = GPT2LMHeadModel.from_pretrained(model_type)
        sd_hf = model_hf.state_dict()

        # copy while ensuring all of the parameters are aligned and match in names and shapes
        sd_keys_hf = sd_hf.keys()
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.masked_bias')] # ignore these, just a buffer
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.bias')] # same, just the mask (buffer)
        transposed = ['attn.c_attn.weight', 'attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']
        # basically the openai checkpoints use a "Conv1D" module, but we only want to use a vanilla Linear
        # this means that we have to transpose these weights when we import them
        assert len(sd_keys_hf) == len(sd_keys), f"mismatched keys: {len(sd_keys_hf)} != {len(sd_keys)}"
        for k in sd_keys_hf:
            if any(k.endswith(w) for w in transposed):
                # special treatment for the Conv1D weights we need to transpose
                assert sd_hf[k].shape[::-1] == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k].t())
            else:
                # vanilla copy over the other parameters
                assert sd_hf[k].shape == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k])

        return model

    def configure_optimizers(self, weight_decay, learning_rate, betas, device_type):
        # start with all of the candidate parameters
        param_dict = {pn: p for pn, p in self.named_parameters()}
        # filter out those that do not require grad
        param_dict = {pn: p for pn, p in param_dict.items() if p.requires_grad}
        # create optim groups. Any parameters that is 2D will be weight decayed, otherwise no.
        # i.e. all weight tensors in matmuls + embeddings decay, all biases and layernorms don't.
        decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]
        nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]
        optim_groups = [
            {'params': decay_params, 'weight_decay': weight_decay},
            {'params': nodecay_params, 'weight_decay': 0.0}
        ]
        num_decay_params = sum(p.numel() for p in decay_params)
        num_nodecay_params = sum(p.numel() for p in nodecay_params)
        print(f"num decayed parameter tensors: {len(decay_params)}, with {num_decay_params:,} parameters")
        print(f"num non-decayed parameter tensors: {len(nodecay_params)}, with {num_nodecay_params:,} parameters")
        # Create AdamW optimizer and use the fused version if it is available
        fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
        use_fused = fused_available and device_type == 'cuda'
        extra_args = dict(fused=True) if use_fused else dict()
        optimizer = torch.optim.AdamW(optim_groups, lr=learning_rate, betas=betas, **extra_args)
        print(f"using fused AdamW: {use_fused}")


## Training GPT with Music Data



In [ ]:
# https://github.com/karpathy/nanoGPT/blob/master/train.py
# http://jalammar.github.io/illustrated-gpt2/ - can this even be used for music generation?